# Metadata Explorer

A notebook to explore the metadata so we know where to look for things!

### Import some libraries

In [19]:
import os
import sys
from socket import gethostname

hostname = gethostname()

if hostname.startswith('hpc-node'):
    IN_DEEPTHOUGHT = True
    sys.path.append('..')
else:
    IN_DEEPTHOUGHT = False
from cf_analysis_lib.load_libraries import *
import cf_analysis_lib


### Read the data

In [20]:
sequence_type = 'MGI_minion'
metadata = cf_analysis_lib.read_metadata(datadir, sequence_type)
metadata.head(5)

,minion,MGI,pwCF_ID,Sample date,IP vs OP,Hospital,Room,Age,Age groups,Paediatric vs Adult,...,Sum of meds,Sum of antifungals,Sum of steroid + mabs,DNA_extraction_ conc,SAGC ULN,DNA Conc. (ng/ul),Index I7,Index I5,Mean_Size_BP,Total Clusters Passing Filter (Million)
NAME,,,,,,,,,,,,,,,,,,,,,
1068841_20180306_S,1068841_20180306_S,1068841_20180306_S,1068841,3/06/2018,OP,RAH,Chest Clinic 7,47,7,Adult,...,0,0,0,1.070,SAGCFN_22_01754,42.8,CGCTCATT,ATAGAGGC,417,2.8
1085876_20180227_S,NaN,1085876_20180227_S,1085876,2/27/2018,OP,RAH,Chest Clinic 6,40,6,Adult,...,1,0,0,0.132,SAGCFN_22_01828,19.0,AGCGATAG,GGCTCTGA,674,2.5
1128691_20171206_S,NaN,1128691_20171206_S,1128691,12/06/2017,IP,RAH,RAH 8E 05,47,7,Adult,...,3,0,0,1.750,SAGCFN_22_01748,33.4,TCCGGAGA,GGCTCTGA,535,4.5
1128691_20171218_S,1128691_20171218_S,1128691_20171218_S,1128691,12/18/2017,OP,RAH,Chest Clinic 1,47,7,Adult,...,1,0,0,3.400,SAGCFN_22_01743,30.8,ATTACTCG,CAGGACGT,352,4.8
1128691_20180116_S,1128691_20180116_S,1128691_20180116_S,1128691,1/16/2018,OP,RAH,Chest Clinic 4,47,7,Adult,...,1,0,0,0.708,SAGCFN_22_01772,27.4,ATTCAGAA,GGCTCTGA,500,3.8


In [111]:
len(metadata['pwCF_ID'].unique())

64

## Antibiotics

In [23]:
nw=['P12M', 'N12M', 'CS_', 'O_']
for c in metadata.columns:
    tp = True
    for n in nw:
        if n in c:
            tp = False
    if tp:
        print(c)

minion
MGI
pwCF_ID
Sample date
IP vs OP
Hospital
Room
Age
Age groups
Paediatric vs Adult
Gender
Sample_Type
H2_Uncorrected
CH4_Uncorrected
CO2
H2_Corrected
CH4_Corrected
CH4/H2 ratio_corrected
Corr.
Culture Result (Matched with sequenced sample)
NTM
Pseudomonas Culture
Cutured in previous 12 months
Others cultured
IgE
Spec IgE
Spec IgG
Precipitins
FVC
FEV1
Best FEV1
FEV1/best FEV1
FEV1_Obstruction_Rank
Cystic Fibrosis related diabetes (CFRD)
Pancreatic insufficiency (PI)
CF gene 1
CF gene 2
Notes
CFLD
Next PA Positive Date
Positive for PA within next 12 months
Culture for next sample
Antibiotics (duration)
Antibiotics_YN
IV
INH
PO
2 Amphotericin B (Ambisome)_INH
4 Amikacin_IV
2 Amikacin_INH
2 Amoxicillin & Potassium clavulanate (Aug Duo)_PO
3 Azithromycin_IV
2 Azithromycin_PO
3 Aztreonam_IV
3 Cefopime_IV
4 Cefoxitin_IV
3 Ceftazidime_IV
2 Ceftazidime_INH
1 Cephalexin_PO
2 Ciprofloxacin_PO
2 Clarithromycin_PO
2 Clofazimine PO
4 Colistin_IV
2 Colistin_IHN
1 Flucloaxcillin_PO
3 Imipenem
1 

In [87]:

# Extract clean drug name and route
def parse_column(col):
    m = re.match(r"^(\d+\s*.*?)_?(IV|INH|PO)?$", col)
    if not m:
        return None, None
    drug, route = m.groups()
    return (drug.strip(), route)

parsed = []
good_cols = []
for  c in metadata.columns:
    d = parse_column(c)
    if d[0] and d[1]:
        parsed.append(d)
        good_cols.append(c)

print(parsed)
print()
from collections import defaultdict

drug_routes = defaultdict(dict)
for col, (drug, route) in zip(good_cols, parsed):
    if route:
        drug_routes[drug][route] = col



[('2 Amphotericin B (Ambisome)', 'INH'), ('4 Amikacin', 'IV'), ('2 Amikacin', 'INH'), ('2 Amoxicillin & Potassium clavulanate (Aug Duo)', 'PO'), ('3 Azithromycin', 'IV'), ('2 Azithromycin', 'PO'), ('3 Aztreonam', 'IV'), ('3 Cefopime', 'IV'), ('4 Cefoxitin', 'IV'), ('3 Ceftazidime', 'IV'), ('2 Ceftazidime', 'INH'), ('1 Cephalexin', 'PO'), ('2 Ciprofloxacin', 'PO'), ('2 Clarithromycin', 'PO'), ('2 Clofazimine', 'PO'), ('4 Colistin', 'IV'), ('1 Flucloaxcillin', 'PO'), ('1 Itraconazole (Lozenoc)', 'PO'), ('3 Meropenem', 'IV'), ('3 Methylpredinosolone', 'IV'), ('3 piperacillin sodium, tazobactam sodium (Tazocin)', 'IV'), ('2 prednisolone', 'PO'), ('1 Sulfamethoxazole_trimethoprim (Bactrim)', 'PO'), ('3 tobramycin', 'IV'), ('2 tobramycin', 'INH')]



In [104]:
def parse_drug(d):
    m = re.match(r'^\d+\s+(\w.*)', d)
    return m.groups()[0]

for tple in parsed:
    print(f"{tple[0]}\t{parse_drug(tple[0])}")

2 Amphotericin B (Ambisome)	Amphotericin B (Ambisome)
4 Amikacin	Amikacin
2 Amikacin	Amikacin
2 Amoxicillin & Potassium clavulanate (Aug Duo)	Amoxicillin & Potassium clavulanate (Aug Duo)
3 Azithromycin	Azithromycin
2 Azithromycin	Azithromycin
3 Aztreonam	Aztreonam
3 Cefopime	Cefopime
4 Cefoxitin	Cefoxitin
3 Ceftazidime	Ceftazidime
2 Ceftazidime	Ceftazidime
1 Cephalexin	Cephalexin
2 Ciprofloxacin	Ciprofloxacin
2 Clarithromycin	Clarithromycin
2 Clofazimine	Clofazimine
4 Colistin	Colistin
1 Flucloaxcillin	Flucloaxcillin
1 Itraconazole (Lozenoc)	Itraconazole (Lozenoc)
3 Meropenem	Meropenem
3 Methylpredinosolone	Methylpredinosolone
3 piperacillin sodium, tazobactam sodium (Tazocin)	piperacillin sodium, tazobactam sodium (Tazocin)
2 prednisolone	prednisolone
1 Sulfamethoxazole_trimethoprim (Bactrim)	Sulfamethoxazole_trimethoprim (Bactrim)
3 tobramycin	tobramycin
2 tobramycin	tobramycin


In [105]:
def parse_drug(d):
    m = re.match(r'^\d+\s+(\w.*)', d)
    return m.groups()[0]
    
columns = set()
for tple in parsed:
    columns.add(parse_drug(tple[0]))
summary = pd.DataFrame(index=metadata.index, columns=list(sorted(columns)))
#summary = summary.fillna("")

    
for tple in parsed:
    (drug, route) = tple
    print(f"Drug: {drug} Route: {route}")
    for c in summary.index:
        if f"{drug}_{route}" in metadata.columns:
            if metadata.loc[c, f"{drug}_{route}"] == 1:
                sd = parse_drug(drug)
                if pd.isna(summary.loc[c, sd]):
                    summary.loc[c, sd] = route
                else:
                    summary.loc[c, sd] += f"; {route}"
summary

Drug: 2 Amphotericin B (Ambisome) Route: INH
Drug: 4 Amikacin Route: IV
Drug: 2 Amikacin Route: INH
Drug: 2 Amoxicillin & Potassium clavulanate (Aug Duo) Route: PO
Drug: 3 Azithromycin Route: IV
Drug: 2 Azithromycin Route: PO
Drug: 3 Aztreonam Route: IV
Drug: 3 Cefopime Route: IV
Drug: 4 Cefoxitin Route: IV
Drug: 3 Ceftazidime Route: IV
Drug: 2 Ceftazidime Route: INH
Drug: 1 Cephalexin Route: PO
Drug: 2 Ciprofloxacin Route: PO
Drug: 2 Clarithromycin Route: PO
Drug: 2 Clofazimine Route: PO
Drug: 4 Colistin Route: IV
Drug: 1 Flucloaxcillin Route: PO
Drug: 1 Itraconazole (Lozenoc) Route: PO
Drug: 3 Meropenem Route: IV
Drug: 3 Methylpredinosolone Route: IV
Drug: 3 piperacillin sodium, tazobactam sodium (Tazocin) Route: IV
Drug: 2 prednisolone Route: PO
Drug: 1 Sulfamethoxazole_trimethoprim (Bactrim) Route: PO
Drug: 3 tobramycin Route: IV
Drug: 2 tobramycin Route: INH


,Amikacin,Amoxicillin & Potassium clavulanate (Aug Duo),Amphotericin B (Ambisome),Azithromycin,Aztreonam,Cefopime,Cefoxitin,Ceftazidime,Cephalexin,Ciprofloxacin,...,Clofazimine,Colistin,Flucloaxcillin,Itraconazole (Lozenoc),Meropenem,Methylpredinosolone,Sulfamethoxazole_trimethoprim (Bactrim),"piperacillin sodium, tazobactam sodium (Tazocin)",prednisolone,tobramycin
NAME,,,,,,,,,,,,,,,,,,,,,
1068841_20180306_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1085876_20180227_S,NaN,NaN,NaN,PO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128691_20171206_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,IV,NaN,NaN,NaN,NaN,IV; INH
1128691_20171218_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IV
1128691_20180116_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895293_20180502_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896213_20180427_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IV,NaN,NaN
913873_20180417_S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
summary.loc['698917_20180329_S', :]

Amikacin                                            IV; INH
Amoxicillin & Potassium clavulanate (Aug Duo)           NaN
Amphotericin B (Ambisome)                               NaN
Azithromycin                                             PO
Aztreonam                                               NaN
Cefopime                                                NaN
Cefoxitin                                               NaN
Ceftazidime                                             NaN
Cephalexin                                              NaN
Ciprofloxacin                                           NaN
Clarithromycin                                          NaN
Clofazimine                                             NaN
Colistin                                                NaN
Flucloaxcillin                                          NaN
Itraconazole (Lozenoc)                                  NaN
Meropenem                                               NaN
Methylpredinosolone                     

In [107]:
summna = summary.dropna(how='all', axis=0)
summna = summna.dropna(how='all', axis=1)

summna.fillna("")
summna.to_csv("antibiotics.tsv", sep="\t")

## What other Cultures do we have

In [18]:
for c in sorted(metadata.columns):
    if c.startswith('CS'):
        print(f"{c.replace('CS_', '')} | {metadata[c].dropna(axis=0).astype(int).sum()}")

Achromobacter xylosoxidans | 3
Acremonium species | 1
Aspergillus flavus | 3
Aspergillus fumigatus | 22
Aspergillus nidulans | 2
Aspergillus niger | 1
Aspergillus terreus | 1
Burkholderia cepacia | 1
Candida albicans | 40
Haemophilus influenzae | 1
Inquilinus limosus | 1
Lomentospora prolificans | 1
MAC_(Smear negative) | 2
MDR_Pseudomonas aeruginosa | 14
Mycobacterium intracellulare | 1
Mycobacteroides abscessus | 6
Mycolicibacter terrae | 1
NTM_(Smear negative) | 1
Oral flora | 31
Penicillium | 3
Pseudomonas aeruginosa | 36
Scedosporium apiospermum | 0
Scedosporium apiospermum.1 | 1
Staphylococcus  aureus | 28
Stenophotomonas maltophilia | 9
mucoid | 19
non-mucoid | 13


# Who has _Pseudomonas_

In [3]:
metadata[metadata['CS_Pseudomonas aeruginosa'] == 0].shape

(91, 166)

In [4]:
len(metadata['pwCF_ID'].unique())

64

In [5]:
len(metadata[metadata['CS_Pseudomonas aeruginosa'] == 0]['pwCF_ID'].unique())

44

In [6]:
nop = set(metadata[metadata['CS_Pseudomonas aeruginosa'] == 0]['pwCF_ID'].unique())
yep = set(metadata[metadata['CS_Pseudomonas aeruginosa'] == 1]['pwCF_ID'].unique())

In [7]:
nop.intersection(yep)

{650003, 658355, 676138, 698564, 715927, 748160, 748699, 788707}

### Get the unique people, and only choose the first sampling date

In [8]:
idx = metadata.groupby("pwCF_ID")['Sample date'].idxmin()
um = metadata.loc[idx]
um

,minion,MGI,pwCF_ID,Sample date,IP vs OP,Hospital,Room,Age,Age groups,Paediatric vs Adult,...,Sum of meds,Sum of antifungals,Sum of steroid + mabs,DNA_extraction_ conc,SAGC ULN,DNA Conc. (ng/ul),Index I7,Index I5,Mean_Size_BP,Total Clusters Passing Filter (Million)
NAME,,,,,,,,,,,,,,,,,,,,,
623361_20180123_S,623361_20180123_S,623361_20180123_S,623361,1/23/2018,OP,RAH,Chest Clinic 9,18,4,Adult,...,0,0,0,0.778,SAGCFN_22_01764,33.60,GAGATTCC,GGCTCTGA,468,2.4
634207_20180510_S,NaN,634207_20180510_S,634207,5/10/2018,IP,WCH,Adol Rm9,17,3,Paediatric,...,1,0,0,0.000,SAGCFN_22_01856,7.82,CGGACGATTC,CCACCACCTA,651,2.9
639354_20171206_S,639354_20171206_S,639354_20171206_S,639354,12/06/2017,IP,WCH,Adolescent 10,17,3,Paediatric,...,1,0,0,3.540,SAGCFN_22_01742,20.80,ATTACTCG,TAATCTTA,337,4.9
642660_20180601_S,642660_20180601_S,642660_20180601_S,642660,6/01/2018,IP,WCH,Adol Rm 11,17,3,Paediatric,...,2,0,0,0.538,SAGCFN_22_01780,21.80,GAATTCGT,GGCTCTGA,491,4.8
650003_20180207_S,650003_20180207_S,650003_20180207_S,650003,2/07/2018,IP,WCH,Adol Room 1,17,3,Paediatric,...,5,0,0,0.552,SAGCFN_22_01778,8.28,GAATTCGT,ATAGAGGC,476,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664053_20180406_S,NaN,1664053_20180406_S,1664053,4/06/2018,OP,RAH,Chest Clinic 1,26,5,Adult,...,3,1,0,0.159,SAGCFN_22_01822,30.80,TCTCGCGC,TAATCTTA,374,1.9
1690154_20180406_S,NaN,1690154_20180406_S,1690154,4/06/2018,OP,RAH,Chest Clinic 4,25,5,Adult,...,0,0,0,0.276,SAGCFN_22_01804,35.00,CGGCTATG,GGCTCTGA,648,3.1
1834617_20180501_S,1834617_20180501_S,1834617_20180501_S,1834617,5/01/2018,OP,RAH,Chest Clinic 8,21,4,Adult,...,1,0,0,0.430,SAGCFN_22_01790,17.00,CTGAAGCT,TAATCTTA,545,4.8


In [9]:
Q1 = um['Age'].quantile(0.25)
Q3 = um['Age'].quantile(0.75)
print(f"Median: {um['Age'].median()} and IQR = {Q3 - Q1} ({Q1}-{Q3})")

Median: 16.5 and IQR = 15.5 (11.0-26.5)


In [10]:
um[um['Age'] >= 18].shape

(27, 166)

In [11]:
print(f"First sample date: {metadata['Sample date'].min()} and last: {metadata['Sample date'].max()}")
print(f"Number of days between = {metadata['Sample date'].max() - metadata['Sample date'].min()}")

First sample date: 1/01/2019 and last: 9/27/2017


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
sum(metadata[['pwCF_ID']].value_counts() > 1)

In [ ]:
for col in metadata.columns:
    counts = um[col].value_counts()
    if len(counts) > 3:
        print(f"{col}\t{len(counts)} values")
    else:    
        summary = "; ".join([f"{cat}: {count}" for cat, count in counts.items()])
        print(f"{col}\t{summary}")

In [ ]:
for col in metadata.columns:
    counts = metadata[col].value_counts()
    if len(counts) > 3:
        print(f"{col}\t{len(counts)} values")
    else:    
        summary = "; ".join([f"{cat}: {count}" for cat, count in counts.items()])
        print(f"{col}\t{summary}")

In [ ]:
metadata['DNA_extraction_ conc']

In [ ]:
metadata['CS_Inquilinus limosus'].dtypes

### 


In [ ]:
for col in metadata.select_dtypes(include='int64').columns:
    counts = metadata[col].value_counts()
    if len(counts) > 3:
        # print(f"{col}\t{len(counts)} values")
        continue
    else:
        #summary = "; ".join([f"{cat}: {count}" for cat, count in counts.items()])
        if int(1) in counts:
            summary = f"{counts[int(1)]}"
        elif int(0) in counts and counts[int(0)] == 127:
            print(f"{col}\tNo positives")
        else:
            summary = "; ".join([f"{cat}: {count}" for cat, count in counts.items()])
        print(f"{col}\t{summary}")
    

In [ ]:
for c in metadata.columns:
    if 'pseudomon' in c.lower():
        print(c)

In [ ]:
metadata[['CS_Pseudomonas aeruginosa', 'CS_MDR_Pseudomonas aeruginosa', 'O_Pseudomonas aeruginosa (multi-resistant)', 'O_Pseudomonas aeruginosa (non-mucoid)', 'O_Pseudomonas aeruginosa (mucoid)']]

In [ ]:
metadata[metadata['CS_MDR_Pseudomonas aeruginosa'] == 1][['CS_Pseudomonas aeruginosa', 'CS_MDR_Pseudomonas aeruginosa']]

In [ ]:
metadata[metadata['CS_Pseudomonas aeruginosa'] == 1][['CS_Pseudomonas aeruginosa', 'CS_MDR_Pseudomonas aeruginosa']].shape

In [ ]:
t = '"; "'.join(sorted(list(metadata.columns)))
print(f'"{t}"')

In [ ]:
antibiotics = ['1 Cephalexin_PO', '1 Flucloaxcillin_PO', '1 Itraconazole (Lozenoc)_PO', '1 Sulfamethoxazole_trimethoprim (Bactrim)_PO', '2 Amikacin_INH', '2 Amoxicillin & Potassium clavulanate (Aug Duo)_PO', '2 Amphotericin B (Ambisome)_INH', '2 Azithromycin_PO', '2 Ceftazidime_INH', '2 Ciprofloxacin_PO', '2 Clarithromycin_PO', '2 Clofazimine PO', '2 Colistin_IHN', '2 prednisolone_PO', '2 tobramycin_INH', '3 Azithromycin_IV', '3 Aztreonam_IV', '3 Cefopime_IV', '3 Ceftazidime_IV', '3 Imipenem', '3 Ivacaftor (Kalydeco)', '3 Meropenem_IV', '3 Methylpredinosolone_IV', '3 Omalizumab_SC', '3 piperacillin sodium, tazobactam sodium (Tazocin)_IV', '3 tobramycin_IV', '4 Amikacin_IV', '4 Cefoxitin_IV', '4 Colistin_IV']

In [ ]:
t = '"; "'.join(sorted(antibiotics))
print(f'"{t}"')

In [ ]:
combinations = {
'Amikacin' : ['2 Amikacin_INH', '4 Amikacin_IV'],
'Azithromycin' : ['2 Azithromycin_PO', '3 Azithromycin_IV'],
'Tobramycin' : ['2 tobramycin_INH', '3 tobramycin_IV'],
'Colistin' : ['2 Colistin_IHN', '4 Colistin_IV'],
'Ceftazidime' : ['2 Ceftazidime_INH', '3 Ceftazidime_IV'],
'Prednisolone' : ['2 prednisolone_PO', '3 Methylpredinosolone_IV']
}

In [ ]:
metadata[(metadata['2 Amikacin_INH'] == 1) | (metadata['4 Amikacin_IV'] == 1)]

In [ ]:
tmpdf = pd.DataFrame()
for new_col, col_array in combinations.items():
    tmpdf[new_col] = metadata[col_array].any(axis=1).astype(int)
tmpdf

In [ ]:
tmpdf.sum()

In [ ]:
len(metadata['pwCF_ID'].unique())

Generate some statistics about our samples

In [ ]:
sequence_type='MGI'

for taxa in ['kingdom', 'order', 'phylum', 'class', 'family', 'genus']:
    genus_otu = cf_analysis_lib.read_taxonomy(datadir, sequence_type, taxa)
    genus_otu = genus_otu.T
    print(f"The taxonomy df for {taxa} has shape: {genus_otu.shape}")

metadata = cf_analysis_lib.read_metadata(datadir, sequence_type, categorise=True)
print(f"The metadata df has shape: {metadata.shape}")


In [ ]:
for taxa in ['kingdom', 'order', 'phylum', 'class', 'family', 'genus']:
    genus_otu = cf_analysis_lib.read_taxonomy(datadir, sequence_type, taxa)
    genus_otu = genus_otu.T
    print(f"we have {genus_otu.shape[1]} taxonomic group for {taxa}, and ", end="")
print()

In [ ]:
for sslevel in ['all_norm_ss.tsv.gz', 'class_norm_ss.tsv.gz', 'level1_norm_ss.tsv.gz', 'level2_norm_ss.tsv.gz', 'subsystems_norm_ss.tsv.gz']:
    ss_df = cf_analysis_lib.read_subsystems(os.path.join(datadir, sequence_type, "FunctionalAnalysis", "subsystems", sslevel), sequence_type)
    ss_df = ss_df.T
    print(f"we have {ss_df.shape[1]} features for {sslevel.replace('_norm_ss.tsv.gz', '')} subsystems, and ", end="")
print()

In [ ]:
sslevel = 'all_norm_ss.tsv.gz'
ss_df = cf_analysis_lib.read_subsystems(os.path.join(datadir, sequence_type, "FunctionalAnalysis", "subsystems", sslevel), sequence_type)
ss_df = ss_df.T
print(f"The subsystems df for {sslevel} has shape: {ss_df.shape}")

In [ ]:
os.path.join(datadir, sequence_type, "FunctionalAnalysis", "subsystems", sslevel)

In [ ]:
intcol = 'CFLD'

lens = metadata[intcol].count()
noss = metadata[intcol].str.contains('No').sum()
yess = metadata[intcol].str.contains('Yes').sum()
print(f"No: {noss} Yes: {yess} tot: {noss+yess} len: {lens}")

In [ ]:
metadata[intcol].dropna().cat.categories

In [ ]:
tmp = metadata[metadata[intcol] != 'Unknown'].dropna()
tmp[intcol]

In [ ]:
mask = metadata[intcol].notna() & (metadata[intcol] != 'Unknown')
tmpdf = metadata[mask]
tmpdf[tmpdf[intcol] == 'Unknown']

In [ ]:
for c in sorted(metadata.columns):
    if 'CS' in c:
        print(c)

In [ ]:
for intcol in sorted(metadata.columns):
    if 'CS' in intcol:
        try:
            mn = metadata.groupby(intcol).size()[1]
        except KeyError as e:
            mn = "NOT KNOWN"
        print(f"{intcol}XXX{mn} pwCF")

In [ ]:
intcol = 'CS_Aspergillus fumigatus'
metadata[intcol].values

In [ ]:
intcol = 'CO2'
metadata[intcol].values

In [ ]:
taxa = 'family'
genus_otu = cf_analysis_lib.read_taxonomy(datadir, sequence_type, taxa, all_taxa=True)
genus_otu = genus_otu.T
genus_otu

In [ ]:
for c in genus_otu.columns:
    if 'Asper' in c:
        print(c)

In [ ]:
genus_otu.loc['650003_20180207_S', 'Aspergillaceae']

In [ ]:
aspergs = []
for c in metadata.columns:
    if c.startswith('CS_Aspergillus'):
        aspergs.append(c)
aspergs

In [ ]:
metadata.loc['650003_20180207_S', aspergs]

In [ ]:
metadata.loc['676138_20180405_S', aspergs]

In [ ]:
metadata[metadata['pwCF_ID']==676138].loc[:,['pwCF_ID', 'Sample date', 'Sum of antifungals'] + aspergs].sort_values(by='Sample date')

In [ ]:
for c in metadata.columns:
    if 'fungal' in c.lower():
        print(c)

In [ ]:
for c in metadata.columns:
    if 'mucoid' in c.lower():
        print(c)

In [ ]:
metadata[metadata['CS_Pseudomonas aeruginosa'] == 1][['Pseudomonas Culture', 'CS_Pseudomonas aeruginosa', 'CS_mucoid', 'CS_non-mucoid']]

In [ ]:
for f in metadata.index:
    if metadata.loc[f, 'Pseudomonas Culture'] != metadata.loc[f, 'CS_Pseudomonas aeruginosa']:
        print(metadata.loc[f, ['Pseudomonas Culture', 'CS_Pseudomonas aeruginosa']])

In [ ]:
metadata[(metadata['N12M_Pseudomonas aeruginosa (multi-resistant)'] != 1) & (metadata['N12M_Pseudomonas aeruginosa'] == 1)][['N12M_Pseudomonas aeruginosa (multi-resistant)', 'N12M_Pseudomonas aeruginosa']]

In [ ]:
for c in metadata.columns:
    if 'fev' in c.lower() or 'fvc' in c.lower():
        print(c)

In [ ]:
sns.scatterplot(data=metadata, x='FEV1_RATIO_SCORE', y='FEV1/best FEV1')

In [ ]:
sns.scatterplot(data=metadata, x='FEV1_Obstruction_Rank', y='FVC')

In [ ]:
tmp = metadata.copy()
tmp['FEV/FVC'] = tmp['FEV1']/tmp['FVC']

In [ ]:
sns.scatterplot(data=tmp, x='FEV1_Obstruction_Rank', y='FEV/FVC')

In [ ]:
for c in metadata.columns:
    if 'CS_Inquilinus' in c:
        print(f"'{c}'")

In [ ]:
metadata[metadata['CS_Inquilinus limosus'] == 1]

In [ ]:
metadata.groupby(by='Gender').count()